In [1]:
import pandas as pd

In [2]:
campione = pd.read_excel("./campione_400.xlsx")
print("Numero Coppie nel Campione:", len(campione))
print("Numero Coppie 'Matching' nel Campione:", len(campione[campione['Label'] == 1]))
print("Numero Coppie 'Non Matching' nel Campione:", len(campione[campione['Label'] == 0]))

Numero Coppie nel Campione: 400
Numero Coppie 'Matching' nel Campione: 173
Numero Coppie 'Non Matching' nel Campione: 227


# Valutazione del Blocking con Word2Vec
Calcoleremo Precision e Recall utilizzando il seguente formalismo:
- True Negative -> Coppie 'Non Matching' che non finiscono nello stesso Blocco
- False Positive -> Coppie 'Non Matching' che finiscono nello stesso Blocco
- False Negative -> Coppie 'Matching' che non finiscono nello stesso Blocco
- True Positive -> Coppie 'Matching' che finiscono nello stesso Blocco

In [3]:
schema_mediato_con_blocks = pd.read_csv("./../datasets/MediatedSchemaConBlocks_W2V.csv")

In [4]:
tn, fp, fn, tp = list(), list(), list(), list()

for i in range(0, len(campione)):
    row = campione.iloc[i]
    first, second, true_label = row['Riga_SX'], row['Riga_DX'], row['Label']

    first_block = schema_mediato_con_blocks.iloc[first]['block']
    second_block = schema_mediato_con_blocks.iloc[second]['block']

    computed_label = None
    if first_block == second_block:
        computed_label = 1
    else:
        computed_label = 0
    
    if true_label == 0 and computed_label == 0:
        tn.append((first, second))
    if true_label == 0 and computed_label == 1:
        fp.append((first, second))
    if true_label == 1 and computed_label == 0:
        fn.append((first, second))
    if true_label == 1 and computed_label == 1:
        tp.append((first, second))

In [5]:
recall = len(tp) / (len(tp) + len(fn))
precision = len(tp) / (len(tp) + len(fp))
f_measure = (2 * precision * recall) / (precision + recall)

print("Recall:", recall)
print("Precision:", precision)
print("F-Measure:", f_measure)

Recall: 0.12716763005780346
Precision: 0.6111111111111112
F-Measure: 0.21052631578947367


# Valutazione del Pairwise Matching
Calcoleremo Precision e Recall utilizzando il seguente formalismo:
- True Negative -> Coppie 'Non Matching' che non finiscono nello stesso Clusterino
- False Positive -> Coppie 'Non Matching' che finiscono nello stesso Clusterino
- False Negative -> Coppie 'Matching' che non finiscono nello stesso Clusterino
- True Positive -> Coppie 'Matching' che finiscono nello stesso Clusterino

In [63]:
tn1, fp1, fn1, tp1 = list(), list(), list(), list()

In [61]:
results = dict()

# Apre il file in modalità di lettura
with open("./../3 - Pairwise Matching/results_clustersW2V.txt", 'r') as file:
    # Legge le righe del file
    lines = file.readlines()

i = 0
for line in lines:
    sep_index = line.find(':')
    key = int(line[:sep_index])
    
    if i != len(lines)-1:
        value = eval(line[sep_index+2:-1])
    else:
        value = eval(line[sep_index+2:])
    
    results[key] = value
    i += 1

In [64]:
## DA ULTIMARE!